# 02. Explore loop census
## Project: Bicycle node network loop analysis

This notebook explores the loop census created in notebook 01, beyond descriptive network statistics.

Contact: Michael Szell (michael.szell@gmail.com)

Created: 2024-01-29  
Last modified: 2024-02-19  

## To do

## Parameters

In [ ]:
loop_numnode_bound = 30
scenario = "scenario1"

restrictions = {"scenario1": {"looplengthmin": 5000, "looplengthmax": 20000, "maxslope": 4}, # family with small children
                "scenario2": {"looplengthmin": 10000, "looplengthmax": 40000, "maxslope": 6} # teenage/adult leisure cyclist
                }

PATH = {
    "data_in_network": "../data/input/faxe/network/",
    "data_in_pois": "../data/input/faxe/pois/",
    "data_out": "../data/processed/faxe/",
    "plot": "../plots/faxe/"
}

## Imports

In [ ]:
import geopandas as gpd
import igraph as ig
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from functools import reduce
import pickle 
from itertools import compress
from random import seed, random

## Functions

In [ ]:
def NormalizeData(data):
    return list((data - np.min(data)) / (np.max(data) - np.min(data)))

def getLayout(G, nodes_id, nodes_coords):
    named_vertex_list = G.vs()["name"]
    layout = []
    for n in named_vertex_list:
        pos = nodes_id.index(n)
        layout.append(nodes_coords[pos])
    return layout

def plotCheck(G, nodes_id, nodes_coords, vertex_size=7, edge_color=None):
    fig, ax = plt.subplots()
    layout = getLayout(G, nodes_id, nodes_coords)
    if edge_color is None:
        ig.plot(G, target=ax, vertex_size=vertex_size, layout=layout, vertex_color="green");
    else:
        ig.plot(G, target=ax, vertex_size=vertex_size, layout=layout, edge_color=edge_color, vertex_color="green");
    return fig
    
def getLoopLength(c):
    l = 0
    cl = len(c)
    for i in range(cl):
        l += Gnx[c[i%cl]][c[(i+1)%cl]]["weight"]
    return l

## Exploration

### Load data

In [ ]:
with open(PATH['data_out'] + 'loopcensus_'+str(loop_numnode_bound)+'.pkl', 'rb') as f:
    allloops = pickle.load(f)
    alllooplengths = pickle.load(f)
    allloopnumnodes = pickle.load(f)
    allloopmaxslopes = pickle.load(f)
    G = pickle.load(f)
    loop_numnode_bound = pickle.load(f)
    nodes_id = pickle.load(f)
    nodes_coords = pickle.load(f)
    numloops = pickle.load(f)
    loopbasis = pickle.load(f)

In [ ]:
# Plot to double-check
plotCheck(G, nodes_id, nodes_coords);

In [ ]:
fig = plt.figure(figsize=(8, 3))
axes1 = fig.add_axes([0.1, 0.1, 0.35, 0.8])
axes2 = fig.add_axes([0.55, 0.1, 0.35, 0.8])

axes1.hist(alllooplengths, density=True)
axes1.set_xlabel('Length [m]')
axes1.set_ylabel('Probability')
axes1.set_title('Loop lengths')

axes2.hist(allloopnumnodes, density=True, bins=list(range(loop_numnode_bound+1)))
axes2.set_xlabel('Nodes')
axes2.set_title('Nodes per loop')
axes2.set_xlim([0, loop_numnode_bound+0.5])

plt.text(loop_numnode_bound/20,0.01, "Bound: " + str(loop_numnode_bound))
plt.text(loop_numnode_bound/20,0.04, "Loops: " + str(numloops));

### Restrict to length bounds

Go through all nodes and restrict their loops to the scenario's length restrictions

In [ ]:
allloops_bound = {}
for sourcenode in allloops:
    this_lengths = np.array(allloops[sourcenode]['lengths'])
    mask_this = (this_lengths >= restrictions[scenario]["looplengthmin"]) & (this_lengths <= restrictions[scenario]["looplengthmax"])
    allloops_bound[sourcenode] = {"loops": list(compress(allloops[sourcenode]['loops'], mask_this)),
                                   "lengths": list(compress(allloops[sourcenode]['lengths'], mask_this)),
                                   "numnodes": list(compress(allloops[sourcenode]['numnodes'], mask_this))}

In [ ]:
vertex_sizes=[len(allloops_bound[k]['loops']) for k in range(len(allloops_bound.keys()))]
numloops_max = max(vertex_sizes)
vertex_sizes = [i/(numloops_max/20) for i in vertex_sizes]

In [ ]:
fig = plotCheck(G, nodes_id, nodes_coords, vertex_size = vertex_sizes)
plt.text(0,0.04, "Max # of loops: " + str(numloops_max));
plt.tight_layout()

In [ ]:
fig.savefig(PATH["plot"] + "network_numloops_"+str(restrictions[scenario]["looplengthmin"])+"to"+str(restrictions[scenario]["looplengthmax"]))

### Restrict to acceptable gradients

In [ ]:
allloops_bound = {}
for sourcenode in allloops:
    this_lengths = np.array(allloops[sourcenode]['lengths'])
    this_maxslopes = np.array(allloops[sourcenode]['max_slopes'])
    mask_this = (this_lengths >= restrictions[scenario]["looplengthmin"])
    mask_this &= (this_lengths <= restrictions[scenario]["looplengthmax"]) 
    mask_this &= (this_maxslopes <= restrictions[scenario]["maxslope"])
    allloops_bound[sourcenode] = {"loops": list(compress(allloops[sourcenode]['loops'], mask_this)),
                                   "lengths": list(compress(allloops[sourcenode]['lengths'], mask_this)),
                                   "numnodes": list(compress(allloops[sourcenode]['numnodes'], mask_this))}

In [ ]:
vertex_sizes=[len(allloops_bound[k]['loops']) for k in range(len(allloops_bound.keys()))]
numloops_max = max(vertex_sizes)
vertex_sizes = [i/(numloops_max/20) for i in vertex_sizes]

In [ ]:
edge_colors = []
for e in G.es:
    if e["max_slope"] > restrictions[scenario]["maxslope"]:
        edge_colors.append("red")
    else:
        edge_colors.append("grey")

In [ ]:
fig = plotCheck(G, nodes_id, nodes_coords, vertex_size=vertex_sizes, edge_color=edge_colors)
plt.text(0,0.04, "Max # of loops: " + str(numloops_max));
plt.tight_layout()

In [ ]:
fig.savefig(PATH["plot"] + "network_numloops_"+str(restrictions[scenario]["looplengthmin"])+"to"+str(restrictions[scenario]["looplengthmax"])+"_maxslope"+str(restrictions[scenario]["maxslope"]))